# AQC System - GPU Acceleration Verification & Benchmark

This notebook verifies the GPU acceleration features of the AQC system and compares performance against CPU execution.

**Runtime Type**: Ensure you have selected a GPU runtime (Runtime > Change runtime type > T4 GPU).

In [ ]:
# 1. Environment Setup & GPU Check
!nvidia-smi
import os
import sys
import time
import subprocess
import json
import glob

In [ ]:
# 2. Install FFmpeg (if not present)
!ffmpeg -version | head -n 1

In [ ]:
# 3. Setup Codebase (Method A: Direct Upload)
# If you uploaded 'AQC_System_Colab.zip' to the sidebar, run this:
import os
if os.path.exists("AQC_System_Colab.zip"):
    !unzip -o AQC_System_Colab.zip
    print("Files unzipped.")
else:
    print("Zip file not found. If you used Google Drive, skip this step.")

In [ ]:
# 4. Install Requirements
!pip install -r requirements.txt

## Performance Benchmark
We will run the analysis twice:
1.  **CPU Mode** (`--hwaccel none`)
2.  **GPU Mode** (`--hwaccel cuda`)

Then we will compare the total execution duration.

In [ ]:
# 5. Run CPU Benchmark
VIDEO_FILENAME = "test_video.mp4"
print("Starting CPU Analysis...")
start_cpu = time.time()

result_cpu = subprocess.run(
    ["python", "main.py", "--input", VIDEO_FILENAME, "--outdir", "test_output_cpu", "--mode", "strict", "--hwaccel", "none"],
    stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
)

end_cpu = time.time()
dur_cpu = end_cpu - start_cpu

if result_cpu.returncode == 0:
    print(f"[CPU] Success. Duration: {dur_cpu:.2f}s")
else:
    print(f"[CPU] Failed. Duration: {dur_cpu:.2f}s")
    print(result_cpu.stdout)
    print(result_cpu.stderr)

In [ ]:
# 6. Run GPU Benchmark
print("Starting GPU Analysis...")
start_gpu = time.time()

result_gpu = subprocess.run(
    ["python", "main.py", "--input", VIDEO_FILENAME, "--outdir", "test_output_gpu", "--mode", "strict", "--hwaccel", "cuda"],
    stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
)

end_gpu = time.time()
dur_gpu = end_gpu - start_gpu

if result_gpu.returncode == 0:
    print(f"[GPU] Success. Duration: {dur_gpu:.2f}s")
else:
    print(f"[GPU] Failed. Duration: {dur_gpu:.2f}s")
    print(result_gpu.stdout)
    # Check if failure is due to missing codec, fallback to print error
    print(result_gpu.stderr)

In [ ]:
# 7. Generate Comparison Report

print("="*40)
print("       PERFORMANCE REPORT       ")
print("="*40)
print(f"CPU Time : {dur_cpu:.2f}s")
print(f"GPU Time : {dur_gpu:.2f}s")

if dur_gpu < dur_cpu:
    saved = dur_cpu - dur_gpu
    pct = (saved / dur_cpu) * 100
    print(f"Improvement: {saved:.2f}s ({pct:.1f}% faster)")
else:
    print("No improvement detected (System overhead might outweigh decoding gains for small files)")

print("="*40)

In [ ]:
# 8. Sanity Check: Look for Hardware Activity in Logs
# We scan the GPU logs/output for keywords if possible, or just trust the subprocess return.
# Since we captured stdout, we can check that.

if "ACCEL" in result_gpu.stdout or "cuda" in result_gpu.stderr.lower() or "nvdec" in result_gpu.stderr.lower():
    print("✅ GPU Hardware Acceleration Usage CONFIRMED in logs.")
else:
    print("⚠️ GPU usage not explicitly found in captured logs (Enable more verbose logging if needed).")